In [33]:
import pandas as pd
import altair as alt

In [34]:
mit = pd.read_csv("Mit UI.csv")
ohne = pd.read_csv("Ohne UI.csv")
demo = pd.read_csv("Demografische Daten.csv")

In [35]:
mit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 35 columns):
 #   Column                                                                                         Non-Null Count  Dtype
---  ------                                                                                         --------------  -----
 0   Kürzel                                                                                         6 non-null      int64
 1   Es war einfach zu verstehen, wie Aktionen im Spiel auszuführen sind.                           6 non-null      int64
 2   Die Aktionen zur Spielsteuerung waren für mich klar                                            6 non-null      int64
 3   Ich fand das Spiel war leicht zu bedienen                                                      6 non-null      int64
 4   Ich habe das Gesamtziel des Spiels verstanden                                                  6 non-null      int64
 5   Die Ziele des Spiels waren für mich klar

In [36]:
long_name = mit.columns[1]

temp_df = mit[[long_name]].copy()
temp_df.columns = ['score']
chart = alt.Chart(temp_df).mark_boxplot(extent='min-max').encode(
    y=alt.Y('score:Q',
            scale=alt.Scale(domain=[3, 6]), # Force scale to show 1-6
            title="Antwort-Skala")
).properties(
    width=200,
    title=f"{long_name}..."
)

chart.display()


alt.Chart(...)

In [37]:
# 1. Select the columns you want to compare (e.g., index 1 to 6)
# We use .iloc to select by index position
cols_to_compare = mit.columns[1:]
subset = mit[cols_to_compare]

# 2. "Melt" the data: This turns columns into rows
# 'var_name' becomes the new column for the Questions
# 'value_name' becomes the new column for the 1-6 Scores
melted_df = subset.melt(var_name='Frage', value_name='Score')

# 3. Create the multi-column Boxplot
chart = alt.Chart(melted_df).mark_boxplot(extent='min-max').encode(
    x=alt.X('Frage:N',
            axis=alt.Axis(labels=False), # Hides the long sentences on the X-axis
            title="Fragen (Vergleich)"),
    y=alt.Y('Score:Q',
            scale=alt.Scale(domain=[0, 6]),
            title="Antwort-Skala (1-6)"),
    color=alt.Color('Frage:N', legend=alt.Legend(title="Fragen-Index", labelLimit=300)),
    tooltip=['Frage', 'Score'] # Hover to see the full question and value
).properties(
    width=500,
    height=400,
    title="Vergleich der Spielerfahrung"
)

chart.display()

alt.Chart(...)

In [38]:
def aggregate_by_catergory(df):# Define your categories
    categories = [
        "EaseOfControl", "GoalsAndRules", "Challange", "ProgressFeedback",
        "AudiovisualAppeal", "Meaning", "Curiosity", "Mastery",
        "Immersion", "Autonomie", "UI"
    ]

    # Create a new empty DataFrame
    df_aggregated = pd.DataFrame()

    # Loop through and average every 3 columns
    for i, cat in enumerate(categories):
        start_col = (i * 3) + 1  # +1 because index 0 is 'Kürzel'

        if cat == "UI":
            # For the last category, we take everything from 31 to the end
            df_aggregated[cat] = df.iloc[:, start_col:].mean(axis=1)
        else:
            # Take exactly 3 columns and average them
            end_col = start_col + 3
            df_aggregated[cat] = df.iloc[:, start_col:end_col].mean(axis=1)

    # Resulting dataframe has 6 rows (one per participant) and 11 category columns
    return df_aggregated

In [39]:
df_aggregated_mit = aggregate_by_catergory(mit)
df_aggregated_ohne = aggregate_by_catergory(ohne)

In [40]:
def visualize_boxplotted(df):
    # 1. Select the columns you want to compare (e.g., index 1 to 6)
    # We use .iloc to select by index position
    cols_to_compare = df.columns[0:]
    subset = df[cols_to_compare]

    # 2. "Melt" the data: This turns columns into rows
    # 'var_name' becomes the new column for the Questions
    # 'value_name' becomes the new column for the 1-6 Scores
    melted_df = subset.melt(var_name='Frage', value_name='Score')

    # 3. Create the multi-column Boxplot
    chart = alt.Chart(melted_df).mark_boxplot(extent='min-max').encode(
        x=alt.X('Frage:N',
                axis=alt.Axis(labels=False), # Hides the long sentences on the X-axis
                title="Fragen (Vergleich)"),
        y=alt.Y('Score:Q',
                scale=alt.Scale(domain=[0, 6]),
                title="Antwort-Skala (1-6)"),
        color=alt.Color('Frage:N', legend=alt.Legend(title="Fragen-Index", labelLimit=300)),
        tooltip=['Frage', 'Score'] # Hover to see the full question and value
    ).properties(
        width=500,
        height=400,
        title="Vergleich der Spielerfahrung"
    )

    chart.display()

In [41]:
visualize_boxplotted(df_aggregated_mit)

alt.Chart(...)

In [42]:
visualize_boxplotted(df_aggregated_ohne)

alt.Chart(...)